# 1. NLP Basic Processing 

In [1]:
import os
import pathlib

ROOT_PATH = pathlib.Path().resolve().parent.parent
print(ROOT_PATH)

os.chdir(ROOT_PATH)

/Users/andrey/PycharmProjects


## Сбор датасета

In [2]:
import glob
import pandas as pd
from typing import (
    List,
    Optional,
)

In [3]:
# Find all files with specified extenstion
def find_all_files(
    pattern: str,
    folder: Optional[pathlib.Path] = None,
) -> List[str]:
    file_path = pattern
    if folder:
        file_path = str(ROOT_PATH / pattern)
    files = glob.glob(file_path)
    print(f'Found {len(files)} files with format txt.')
    return files

FILES_PATTERN = 'data/raw/*.tsv'

files = find_all_files(FILES_PATTERN)

Found 333 files with format txt.


In [5]:
def read_all_files(files: List[str]) -> pd.DataFrame:
    raw_df = pd.DataFrame()
    
    for file_name in files:
        local_df = pd.read_csv(file_name, sep='\t')
        local_df['filename'] = file_name.split('/')[-1]
        raw_df = pd.concat([raw_df, local_df], axis=0)
        
    return raw_df

ungrouped_df = read_all_files(files)
ungrouped_df.head()

,start,end,text,filename
0,0,4700,If you've ever wanted to replicate shots like ...,IMG_0832.tsv
1,4700,9380,"constant aperture, then today is your lucky da...",IMG_0832.tsv
2,9380,14080,TV lenses to your mirrorless camera. So there'...,IMG_0832.tsv
3,14420,17900,"Fujinon and Canon, and both of them have chart...",IMG_0832.tsv
4,17900,22540,which lenses are going to work with your camer...,IMG_0832.tsv


In [6]:
aggregated_texts = ungrouped_df.groupby('filename')['text'].agg(lambda x: ' '.join(x))

agg_df = aggregated_texts.reset_index()

agg_df.head()

,filename,text
0,FullSizeRender.tsv,字幕志愿者 李宗盛
1,IMG_0703.tsv,This is the most dangerous strike in mixed mar...
2,IMG_0704.tsv,"So this puzzle is a little harder, but let's g..."
3,IMG_0705.tsv,I want to talk about the lie of cultural appro...
4,IMG_0706.tsv,"J'cale le pétard entre mes dents Bah ouais, ba..."


In [7]:
# Saving dataset
agg_df.to_parquet('data/aggregated_text.parquet', index=False)

ungrouped_df.to_parquet('data/texts_with_timecodes.parquet', index=False)